In [8]:
%matplotlib qt5
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
Alpha=0.1 #学习速率
Beta=0.9 #对未来奖励的衰减度
Epsilon_start=1#选择随机动作的概率 Epsilon
Epsilon_stop=0.01
decay_rate=0.001
N_Mesh=3
N_States=9
Actions=['left','right','up','down']
Obstacle=[2,5]
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Caculate_coordinate(State):
    X_Axis=0.5+(State-1)%N_Mesh
    Y_Axis=0.5+((State-1)//N_Mesh)
    return X_Axis,Y_Axis
def Initial_Q_Table (N_States,Actions):#以状态数和动作为变量，返回Q表
    Q_Table=np.zeros([N_States,len(Actions)])#初始化Q_表
    Q_Table=pd.DataFrame(Q_Table,index=np.arange(1,N_States+1),columns=Actions)#Q_表定义行列标签
    print ('初始化Q表完成') 
    return Q_Table
#——————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
def Select_Action(State,Q_Table,ep):#根据目前的状态选择动作,返回执行动作
    Epsilon=Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*ep)
    States_Actions=Q_Table.loc[[State],:]#选择目前状态下，取出Q表的目前的状态行,state选择为数字
    if(np.random.rand()<Epsilon or np.all(States_Actions==[0])):#其实np.all(States_Action==[0])也行
        Execute_Action=np.random.choice(Actions)
    else:
        #Q_T=Q_Table.T
        #G=(Q_Table.T).loc[:,[State]]
        Execute_Action=np.array(pd.DataFrame.idxmax((Q_Table.T).loc[:,[State]]))[0]#这里真心累，有毒
    return Execute_Action
#———————————————————————————————————————————————————————————————————————————————————————————————————————————
def Next_State(State,Execute_Action):#根据目前状态个选择的动作,返回即时奖励和新状态
    if(Execute_Action=='left'):
        #目标在右上角，如果Execute_Action为‘left’，不可能到达目标
        if(State%N_Mesh==1):#判断是不是在网格最左边
            State=State
            R=-1
        elif((State-1) in Obstacle):
            State=State
            R=-1
        else:
            State=State-1
            R=0
    elif(Execute_Action=='right'):
        if(State==N_Mesh-1):
            State='End'
            R=10
        elif(State%N_Mesh==0):
            State=State
            R=-1
        elif((State+1) in Obstacle):
            State=State
            R=-1
        else:
            State=State+1
            R=0
    elif(Execute_Action=='up'):  
        if(N_States+1-N_Mesh<=State<=N_States):
            State=State
            R=-1
        elif((State+N_Mesh) in Obstacle):
            State=State
            R=-1
        else:
            State=State+N_Mesh
            R=0
    else:
        if(State-N_Mesh==3):
            State='End'
            R=10
        elif(0<=State<=N_Mesh):
            State=State
            R=-1
        elif((State-N_Mesh) in Obstacle):
            State=State
            R=-1
        else:
            R=0
            State=State-N_Mesh
    return State,R
        
#————————————————————————————————————————————————————————————————————————————————————————————
def Run_Function():
    #——————————————————————————————————————————————————画图程序部分
    
    
    #——————————————————————————————————————————————————
    Q_Table=Initial_Q_Table (N_States,Actions)
    Step=0 
    for episode in np.arange(100) :#跑20回合
        if(episode%20==0):
            print('Q_Table,%d'%episode)
            Q_Table
#         fig=plt.figure()
#         ax=fig.gca()
#         ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
#         ax.set_xticks(np.arange(0,(N_Mesh+1)))
#         ax.set_yticks(np.arange(0,(N_Mesh+1)))
#         plt.grid()
        print('episode is %d' % episode)
        State_=1 #定义起点 
        print("Normal_start:%d"%State_)
        Is_End=False
        while not Is_End:
            A=Select_Action(State_,Q_Table,Step)
            Next_S,R=Next_State(State_,A)
            Q_Table_S_A=Q_Table.loc[[State_],[A]]
            if Next_S!='End':
                D_T=np.array((Q_Table.loc[[Next_S],:]).T)
                Q_Target=R+Beta*D_T.max() #Next_S状态中的可选动作中的最大Q值动作
                Step+=1
            else:
                Q_Target=R
                Step+=1
                Is_End=True #目的是结束循
#————————————————————————————————————————————————————————————————————————————画图程序
#             X_State,Y_State=Caculate_coordinate(State_)
#             if(Next_S)=='End':
#                 H=N_States
#             else:
#                 H=Next_S
#             X_Next_State,Y_Next_State=Caculate_coordinate(H)                      
#             if(Step==0):
#                 plt.scatter(X_State,Y_State)  
#             else:
#                 plt.scatter(X_State,Y_State) 
#                 plt.scatter(X_Next_State,Y_Next_State)
#                 plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
#————————————————————————————————————————————————————————————————————————————
            Q_Table.loc[[State_],[A]]+=Alpha*(Q_Target-Q_Table_S_A)
            State_=Next_S
        print(Step)
        print(Epsilon_stop+(Epsilon_start-Epsilon_stop)*np.exp(-decay_rate*Step))
        #plt.show()
        #print(Q_Table)
    return Q_Table
#————————————————————————————————————————————————————————————————————————————————————————————————————
Q__Table=Run_Function()
Q__Table
Final_Q_Table=np.array(Q__Table)
#—————————————————————————————————————————————————————————————————————————————————————————————————————
def Plot_Final_Graph(Q_TABLE):
    f=open('A.txt','w+')
    f.write(str(Q_TABLE))
    f.close()
    print(Q_TABLE)
    fig=plt.figure()
    ax=fig.gca()
    ax.set(xlim=[0, N_Mesh], ylim=[0, N_Mesh])
    ax.set_xticks(np.arange(0,(N_Mesh+1)))
    ax.set_yticks(np.arange(0,(N_Mesh+1)))
    plt.grid()
    State_Start=1
    State=State_Start
    Step=1
    while(State!=N_Mesh):
        Action=np.argmax((Q_TABLE[State-1]))
        if(Action==0):
             Next_State=State-1
        elif(Action==1):
             Next_State=State+1
        elif(Action==2):
             Next_State=State+N_Mesh
        else:
             Next_State=State-N_Mesh
        Step+=1
        print(Next_State)
        X_State,Y_State=Caculate_coordinate(State)
        X_Next_State,Y_Next_State=Caculate_coordinate(Next_State)
        if(Step==0):
            plt.scatter(X_State,Y_State)  
        else:
            plt.scatter(X_State,Y_State) 
            plt.scatter(X_Next_State,Y_Next_State)
            plt.plot([X_State,X_Next_State],[Y_State,Y_Next_State])
        for J in np.arange(len(Obstacle)):
            plt.scatter((Caculate_coordinate(Obstacle[J]))[0],(Caculate_coordinate(Obstacle[J]))[1],marker="x",s=300)
        State=Next_State
    plt.show()
#——————————————————————————————————————————————————————————————————————————————————————————————————————————
Plot_Final_Graph(Final_Q_Table)

初始化Q表完成
Q_Table,0
episode is 0
Normal_start:1
80
0.923885182923
episode is 1
Normal_start:1
124
0.884546042474
episode is 2
Normal_start:1
157
0.85615701823
episode is 3
Normal_start:1
201
0.819733307238
episode is 4
Normal_start:1
219
0.805288500972
episode is 5
Normal_start:1
277
0.760473952139
episode is 6
Normal_start:1
300
0.743410038475
episode is 7
Normal_start:1
313
0.733937413443
episode is 8
Normal_start:1
322
0.727451208427
episode is 9
Normal_start:1
362
0.699319544163
episode is 10
Normal_start:1
378
0.688378195659
episode is 11
Normal_start:1
392
0.678947072821
episode is 12
Normal_start:1
404
0.670967680056
episode is 13
Normal_start:1
418
0.661778606141
episode is 14
Normal_start:1
427
0.655938916706
episode is 15
Normal_start:1
438
0.648872525029
episode is 16
Normal_start:1
459
0.635596092451
episode is 17
Normal_start:1
474
0.626282180042
episode is 18
Normal_start:1
483
0.62076052514
episode is 19
Normal_start:1
497
0.612269453973
Q_Table,20
episode is 20
Normal_sta